# **class Client**

In [6]:
import torch
import threading
import queue
class Client:
  def __init__(self, client_id,partition1, partition, input_queue=None, data_loader=None):

        self.client_id = client_id
        self.partition = partition
        self.partition1 = partition1
        self.input_queue = input_queue
        self.data_loader = data_loader

  def process_partition1(self , data ):
    data_tensor = torch.tensor(data)
    return self.partition1(data_tensor)


  def process_partition(self , data ):
    data_tensor = torch.tensor(data)
    return self.partition(data_tensor)

  # calculate loss function
  def process_final_partition(self , data):
    print("calculate loss function here")


  def run(self):
    while True:
      if self.input_queue and not self.input_queue.empty():
        data = self.input_queue.get()
        x = data["output"]
        stage = data["stage"]
        source = data["source"]
        destination = data["destination"]
        client_id = data["client_id"]

        if self.client_id ==1 and ( stage !=1 and stage != 4):
          print("you shoulnt be here !!!")
          break

        # handle stage == 1
        if stage == 1 :
          output = self.process_partition1(x)
          stage +=1
          self.input_queue[stage].put({
               "output":output ,
                "stage":stage,
                "source":self.client_id,
                "destination":stage,
                "client_id":client_id
          })
        elif stage !=5:
          output = self.process_partition(x)
          print(f"x value in client[{client_id}]stage:[{stage}]")
          stage +=1
          self.input_queue[stage].put({
                "output":output ,
                "stage":stage,
                "source":self.client_id,
                "destination":stage,
                "client_id":client_id
            })
        else:
            print("final partition")
            self.process_final_partition(x)
      else:
        if self.data_loader:
          inputs , labels = self.data_loader.get_next_batch()
        self.input_queue[self.client_id].put({
            "output":inputs ,
            "stage":1,
            "source":self.client_id,
            "destination":self.client_id,
            "client_id":self.client_id
        })
      time.sleep(0.5)

# **Data Loader Class**

In [9]:
import random
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader as TorchDataLoader

class DataLoader:
    def __init__(self, dataset, batch_size=32, shuffle=True, train_ratio=0.8):
        """
        Initialize the DataLoader with specific configurations.
        """
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.train_ratio = train_ratio

        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        self.dataset_train, self.dataset_test = self.loadDataSet()

        # ایجاد DataLoader برای train و test
        self.train_loader = TorchDataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=self.shuffle)
        self.test_loader = TorchDataLoader(self.dataset_test, batch_size=self.batch_size, shuffle=False)

        self.train_iter = iter(self.train_loader)
        self.test_iter = iter(self.test_loader)
        self.mode = "train"

    def set_mode(self, mode):
        """
        Set the mode to 'train' or 'test'.
        """
        if mode not in ["train", "test"]:
            raise ValueError("Mode must be either 'train' or 'test'")
        self.mode = mode
        # بازنشانی iterator
        if self.mode == "train":
            self.train_iter = iter(self.train_loader)
        else:
            self.test_iter = iter(self.test_loader)

    def loadDataSet(self):
        """
        Load CIFAR-10 dataset.
        """
        trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=self.transform)
        testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=self.transform)
        return trainset, testset

    def getLoader(self):
        """
        Return the appropriate DataLoader based on the mode.
        """
        return self.train_loader if self.mode == "train" else self.test_loader

    def get_next_batch(self):
        """
        Return the next batch of data.
        """
        if self.mode == "train":
            try:
                batch = next(self.train_iter)
            except StopIteration:
                # اگر به انتهای دیتاست رسیدیم، دوباره از ابتدا شروع کن
                self.train_iter = iter(self.train_loader)
                batch = next(self.train_iter)
        else:
            try:
                batch = next(self.test_iter)
            except StopIteration:
                # اگر به انتهای دیتاست رسیدیم، دوباره از ابتدا شروع کن
                self.test_iter = iter(self.test_loader)
                batch = next(self.test_iter)

        return batch


# **test DataLoader Class**

In [11]:
data_loader = DataLoader(dataset=None , batch_size=64)
data_loader.set_mode("train")

batch_count = 0
while True:
  inputs , labels = data_loader.get_next_batch()
  print(f"fetch {batch_count} and input shape is :${labels.shape}")
  batch_count +=1

  if batch_count ==10 :
    break

Files already downloaded and verified
Files already downloaded and verified
fetch 0 and input shape is :$torch.Size([64])
fetch 1 and input shape is :$torch.Size([64])
fetch 2 and input shape is :$torch.Size([64])
fetch 3 and input shape is :$torch.Size([64])
fetch 4 and input shape is :$torch.Size([64])
fetch 5 and input shape is :$torch.Size([64])
fetch 6 and input shape is :$torch.Size([64])
fetch 7 and input shape is :$torch.Size([64])
fetch 8 and input shape is :$torch.Size([64])
fetch 9 and input shape is :$torch.Size([64])


In [2]:
import torch
import torch.nn as nn

# BasicBlock
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out


# **define Partion class**

In [1]:
import torch
import torch.nn as nn

class Partition(nn.Module):
    def __init__(self):
        super(Partition, self).__init__()
        self.layers = self.build_model()

    def build_model(self):
        raise NotImplementedError("Subclasses should implement this method")

    def forward(self, x):
        return self.layers(x)


In [3]:
class Partition1(Partition):
    def build_model(self):
        return nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

class Partition2(Partition):
    def build_model(self):
        return nn.Sequential(
            BasicBlock(64, 64, stride=1),
            BasicBlock(64, 128, stride=2)
        )

class Partition3(Partition):
    def build_model(self):
        return nn.Sequential(
            BasicBlock(128, 256, stride=2),
            BasicBlock(256, 512, stride=2)
        )

class Partition4(Partition):
    def build_model(self):
        return nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(512, 10)
        )


# **Test partitions and basic Block**


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_loader = DataLoader(dataset=None , batch_size=8)
data_loader.set_mode("test")

partition1 = Partition1().to(device)
partition2 = Partition2().to(device)
partition3 = Partition3().to(device)
partition4 = Partition4().to(device)


images, labels = data_loader.get_next_batch()

# عبور دادن ورودی از طریق پارتیشن‌ها
print("Running Partition1...")
output1 = partition1(images)
print(f"Output shape after Partition1: {output1.shape}")

print("Running Partition2...")
output2 = partition2(output1)
print(f"Output shape after Partition2: {output2.shape}")

print("Running Partition3...")
output3 = partition3(output2)
print(f"Output shape after Partition3: {output3.shape}")

print("Running Partition4...")
output4 = partition4(output3)
print(f"Output shape after Partition4: {output4.shape}")

# محاسبه پیش‌بینی نهایی
_, predicted = torch.max(output4, 1)
print(f"Predicted labels: {predicted}")
print(f"True labels: {labels}")

Files already downloaded and verified
Files already downloaded and verified
Running Partition1...
Output shape after Partition1: torch.Size([8, 64, 32, 32])
Running Partition2...
Output shape after Partition2: torch.Size([8, 128, 16, 16])
Running Partition3...
Output shape after Partition3: torch.Size([8, 512, 4, 4])
Running Partition4...
Output shape after Partition4: torch.Size([8, 10])
Predicted labels: tensor([5, 5, 5, 5, 5, 6, 5, 5])
True labels: tensor([3, 8, 8, 0, 6, 6, 1, 6])


# **Message Class**

In [ ]:
class Message:
  def __init__(self , message_type , stage, source , destination,data_id,data):
    self.message_type = message_type
    self.stage = stage
    self.source = source
    self.destination = destination
    self.data_id = data_id
    self.data = data

# **simulate logic Client Running with Thread**

In [2]:
import queue
import time
import threading

class Partition1:
    def process(self, client_id):
        print("partition 1")
        return client_id  #return client id

class Partition2:
    def process(self, x):
        print("partition 2")
        return x + 2

class Partition3:
    def process(self, x):
        print("partition 3")
        return x * 4

class Partition4:
    def process(self, x):
        print("partition 4")
        return x - 1

class FinalPartition:
    def process(self, x, label):
        print(f"Computed Value: {x}, Expected Label: {label}")
        if x == label:
            print("Success!")
        else:
            print("Failed!")



# Label‌ها
labels = {1: 11, 2: 15, 3: 19, 4: 23}

class Client(threading.Thread):
    def __init__(self, client_id, partition1, partition, final_partition  , input_queue ):
        threading.Thread.__init__(self)
        self.client_id = client_id
        self.partition1 = partition1
        self.partition = partition
        self.final_partition = final_partition
        self.input_queue = input_queue
    def run(self):
      while True:
          # check input list
        if not self.input_queue[self.client_id].empty():
          data = self.input_queue[self.client_id].get()
          x = data["output"]
          stage = data["stage"]
          source = data["source"]
          destination = data["destination"]
          client_id = data["client_id"]

          if self.client_id==1 and (stage !=1 and stage !=4):
            print(f'fuck................{stage}')
            break

          # data come from stage4
          # if stage == 4 and client_id==self.client_id:
          #   print(f"final partition for client ${self.client_id}")
          #   self.final_partition.process(x)
          if stage == 1:
            x = self.partition1.process(x)
            stage += 1
            self.input_queue[stage].put({
                "output":x ,
                "stage":stage,
                "source":self.client_id,
                "destination":stage,
                "client_id":client_id
            })
          elif stage!=5:
            x = self.partition.process(x)
            print(f"x value in client[{client_id}]stage:[{stage}]={x}")
            stage +=1
            self.input_queue[stage].put({
                "output":x ,
                "stage":stage,
                "source":self.client_id,
                "destination":stage,
                "client_id":client_id
            })
          else:
            print("final partition")
            self.final_partition.process(x , labels[self.client_id])
        else:
          self.input_queue[self.client_id].put({
                "output":self.client_id ,
                "stage":1,
                "source":self.client_id,
                "destination":self.client_id,
                "client_id":self.client_id})

        time.sleep(5)
      # read from input




def print_queues(queues):
    for key, q in queues.items():
        items = list(q.queue)  # Accessing internal queue list (not thread-safe)
        print(f"Queue {key}: {items}")

In [3]:
import queue

queues = {
    1: queue.Queue(),
    2: queue.Queue(),
    3: queue.Queue(),
    4: queue.Queue(),
    5: queue.Queue()
}
partition1 = Partition1()
partition2 = Partition2()
partition3 = Partition3()
partition4 = Partition4()
final_partition = FinalPartition()


client1 = Client(1, partition1, None, final_partition, queues)
client2 = Client(2, partition1, partition2, final_partition, queues)
client3 = Client(3, partition1, partition3, final_partition, queues)
client4 = Client(4, partition1, partition4, final_partition, queues)

# runinng client parallel
clients = [client1, client2, client3, client4]
for client in clients:
    client.start()

for client in clients:
    client.join()


Exception in thread Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-10Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "<ipython-input-2-d725768aaac4>", line 50, in run
    self.run()
  File "<ipython-input-2-d725768aaac4>", line 50, in run
    self.run()    self.run()
  File "<ipython-input-2-d725768aaac4>", line 50, in run

  File "<ipython-input-2-d725768aaac4>", line 50, in run
TypeError: 'Queue' object is not subscriptable
TypeError: 'Queue' object is not subscriptable
TypeError: 'Queue' object is not subscriptable
TypeError: 

In [ ]:


printQueue()

1
2
3
4


In [ ]:
import queue

# Sample data
queues = {
    1: queue.Queue(),
    2: queue.Queue(),
    3: queue.Queue(),
    4: queue.Queue()
}

# Add some items to the queues
queues[1].put("task1")
queues[2].put("task2")
queues[2].put("task3")
queues[3].put("task4")

# Print the contents of each queue


print_queues(queues)


Queue 1: ['task1']
Queue 2: ['task2', 'task3']
Queue 3: ['task4']
Queue 4: []
